I need to remake the variation excel sheet so that it creates distinct lines for each type of mutation 
and the location that it affects. So if it is intergenic, I have a line for upstream and the downstream gene
or if its in the promoter of one gene and in the coding sequence of another these get seperate lines.
I will need to manually add some things like whether or not the gene is affected or not by the mutation type
whether the innoculum has it 

then for every line I make a promoter region, an downstream region and coding region primers. 

if there is a similar mutation in the region, then the same effect will be predicted e.g. if there 
is an insertion of % 3 != 0 in the innoculum within the coding region, then the gene is off 
if there is not, then the gene is back ON (as in the reference)

if there is a different mutation, i will need to interpret it on the fly i.e. and snv in the coding region will need to be compared by in-silico translation whereas an indel (%3!=0) can be assumed to cause a frameshift, where as ANYthing in a promoter can be interpreted as an "unkown" effect 

I then need to determine whether the homolog in the peruvian genome corresponds to a known gene 
I will need to add the predicted length of the protein and compare it to original 

then I need to go back and determine whether there reference version is functional or not 


BUG TO DO: in Crofts_Relapse_wo_Region for pseD is incorrect? 

FEATURE TO ADD: populations expected not to have the wild type 

Feature to add: see if existing clusters homologs in the abundances in which we detect them 



#### The purpose of this notebook is to analyze the variants reported in Crofts et al. It seeks to reproduce their results with their own data, as well as in recrudescent strains collected from community acquired infections.

#### In this first block, we will simply import the packages we need to analyze the data.

In [1]:
import pandas as pd
import numpy as np
import os
from Bio import SeqIO

#### We will ultimately call SNPs using the library of a relapsed isolate mapped against the draft assembly of a paired primary infection. This differs from the SNP calling pipeline used in the Crofts et al paper becuase they call SNPs against a complete reference genome. To gauge the posibility for false positives and negatives, we simulated a draft assembly of their complete genome using `wgsim`. In this step, we will examine its quality using BLASTn.

#### From this we can see that the reference genome fully covered identically by the assembly. We can also see that there are a number of smaller contigs that are repeatedly aligned with the reference at different points. Some of these repeats stich over portion of the assembly that would be otherwised uncovered (eg. contigs 10 and 12) but others map to regions that are already covered (eg. contig 18). The latter implies that there may be a gene that is repeated, but which has diverged internally, which can aberrations in coverage in the region. 

In [2]:
sim_assem = "/Volumes/KeithSSD/variant-calling/simulated_reference/CG8421_sim_contigs_fixed.fa"
sim_check = "/Volumes/KeithSSD/variant-calling/simulated_reference/simulation_matches.txt"
complete_ref = "/Volumes/KeithSSD/variant-calling/ref_fastas/CG8421_bb.fa"
complete_db = "/Volumes/KeithSSD/variant-calling/simulated_reference/complete_ref_db"

print('\nOUTFMT="6 qseqid sseqid pident length qcovs mismatch gapopen qstart qend qlen sstart send slen evalue bitscore sstrand"\n')
print("makeblastdb -in {} -dbtype nucl -out {}\n".format(complete_ref, complete_db))
bp1 = 'blastn -evalue 1e-3 -db {} -query {} -outfmt '.format(complete_db, sim_assem)
bp2 = '"$OUTFMT" -qcov_hsp_perc 90 -out {} -num_threads 3\n\n'.format(sim_check)
print(bp1+bp2)

blast_cols1 = ['primer', 'sseqid', 'pident', 'length', 'qcov', 'mismatch', 'gapopen', 'qstart', 'qend',
              'qlen', 'sstart', 'send', 'slen', 'evalue', 'bitscore', 'sstrand']
sim_check_df = pd.read_csv(sim_check, sep="\t", header=None)
sim_check_df.columns = blast_cols1
#sim_check_df['reversed'] = sim_check_df[['sstart', 'send']].apply(lambda x: min(x), axis=1)
sim_check_df['sstart2'] = sim_check_df[['sstart', 'send']].apply(lambda x: min(x), axis=1)
sim_check_df['send2'] = sim_check_df[['sstart', 'send']].apply(lambda x: max(x), axis=1)
sim_check_df.sort_values(by='sstart2', inplace=True)

#sim_check_df.loc[[7, 11, 4, 13, 6, 14, 10, 21, 20, 23, 12], ['primer', 'sseqid', 'pident', 'length', 'qcov', 'mismatch', 'gapopen', 'qstart', 'qend',
#              'qlen', 'sstart2', 'send2', 'sstrand']]

sim_check_df.loc[:, ['primer', 'sseqid', 'pident', 'length', 'qcov', 'mismatch', 'gapopen', 'qstart', 'qend',
              'qlen', 'sstart2', 'send2', 'sstrand']]


OUTFMT="6 qseqid sseqid pident length qcovs mismatch gapopen qstart qend qlen sstart send slen evalue bitscore sstrand"

makeblastdb -in /Volumes/KeithSSD/variant-calling/ref_fastas/CG8421_bb.fa -dbtype nucl -out /Volumes/KeithSSD/variant-calling/simulated_reference/complete_ref_db

blastn -evalue 1e-3 -db /Volumes/KeithSSD/variant-calling/simulated_reference/complete_ref_db -query /Volumes/KeithSSD/variant-calling/simulated_reference/CG8421_sim_contigs_fixed.fa -outfmt "$OUTFMT" -qcov_hsp_perc 90 -out /Volumes/KeithSSD/variant-calling/simulated_reference/simulation_matches.txt -num_threads 3




,primer,sseqid,pident,length,qcov,mismatch,gapopen,qstart,qend,qlen,sstart2,send2,sstrand
7,c_000000000008,CG8421-NZ_CP005388-x,100.000,38777,100,0,0,1,38777,38777,1,38777,plus
11,c_000000000010,CG8421-NZ_CP005388-x,99.834,6007,100,0,1,1,6007,6007,38678,44674,plus
4,c_000000000005,CG8421-NZ_CP005388-x,100.000,104606,100,0,0,1,104606,104606,44548,149153,minus
13,c_000000000012,CG8421-NZ_CP005388-x,100.000,1050,100,0,0,1,1050,1050,149027,150076,minus
6,c_000000000007,CG8421-NZ_CP005388-x,100.000,89457,100,0,0,1,89457,89457,150077,239533,minus
14,c_000000000012,CG8421-NZ_CP005388-x,100.000,1050,100,0,0,1,1050,1050,239534,240583,plus
3,c_000000000004,CG8421-NZ_CP005388-x,100.000,153010,100,0,0,1,153010,153010,240457,393466,plus
10,c_000000000010,CG8421-NZ_CP005388-x,99.917,6009,100,3,2,1,6007,6007,393340,399348,plus
1,c_000000000002,CG8421-NZ_CP005388-x,100.000,287092,100,0,0,1,287092,287092,399286,686377,plus
9,c_000000000010,CG8421-NZ_CP005388-x,100.000,6007,100,0,0,1,6007,6007,686251,692257,plus


#### Next, we read in a Genbank file for the reference strain (CG8421) from Crofts et al. We want a dataframe containing gene feature positions, annotations, protein and DNA sequence

In [3]:
gb_file = "/Volumes/KeithSSD/variant-calling/refs/GCF_000171795.2_ASM17179v2_genomic.gbff"

gb_records = {i.name:i for i in list(SeqIO.parse(open(gb_file,"r"), "genbank"))}

gb_subdfs = []
for chrom, gb_record in gb_records.items():
    feat_idx = list(range(len(gb_record.features)))
    exp_cols = set(['molecule', 'type', 'strand', 'start', 'end', 'sequence'])
    exp_cols.update([k for i in feat_idx for k in gb_record.features[i].qualifiers.keys()])
    gb_subdf = pd.DataFrame(index=feat_idx, columns=exp_cols)
    for i in gb_subdf.index:
        feature_i = gb_record.features[i]
        gb_subdf.at[i, 'molecule'] = chrom
        gb_subdf.at[i, 'type'] = feature_i.type
        gb_subdf.at[i, 'strand'] = feature_i.location.strand
        gb_subdf.at[i, 'start'] = int(feature_i.location.start)
        gb_subdf.at[i, 'end'] = int(feature_i.location.end)
        # This already does reverse complementing of crick strand features
        gb_subdf.at[i, 'sequence'] = str(feature_i.extract(gb_record.seq))
        for qk, qv in feature_i.qualifiers.items():
            if len(qv) != 1:
                qv = ", ".join(qv)
            gb_subdf.at[i, qk] = qv[0]
    gb_subdfs.append(gb_subdf.copy())
    print("Parsed {} features from {} to a dataframe size {}".format(len(feat_idx), chrom, gb_subdf.shape))
    
gb_df = pd.concat(gb_subdfs, axis=0, ignore_index=True)
print("Full genbank dataframe sized {}".format(gb_df.shape))


Parsed 3433 features from NZ_CP005388 to a dataframe size (3433, 35)
Full genbank dataframe sized (3433, 35)


#### Next, we will read in some information about the mutations detected in the Crofts study taken from their supplement. This block contains a sanity test whereby we match the reference allele nucleotides specified at locii where deletions and SNVs were discovered can be located in the genome sequence we extracted from our Genbank file. 

#### Positional information about insertions and multinucleotide deletions is given as a string with two coordinates seperated by a carot or two periods. We will reformat this information into a python tuple containing integers so that it can be used to index the string object containing the full genome sequence. 


In [4]:
var_file = '/Volumes/KeithSSD/variant-calling/misc_data/Crofts_Gene_Variants.xlsx'
var_df = pd.read_excel(var_file, sheet_name='Spec_Variants', index_col=None)
var_df['Variant Position'] = var_df['Variant Position'].astype(str).str.replace("\.\.", ',')
var_df['Variant Position'] = var_df['Variant Position'].str.replace("^", ',')

full_genome = gb_df.loc[0, 'sequence']

lociii = []
for var_i in var_df.index:
    locii = str(var_df.loc[var_i, 'Variant Position']).replace("*", "").split(",")
    locii = tuple([int(i) for i in locii])
    lociii.append(locii)
    detectable_muts = var_df.loc[var_i, 'Variant Type'] in ['Deletion', 'SNV']
    if len(locii) == 1 and detectable_muts:
        assert full_genome[locii[0]-1] == var_df.at[var_i, 'Reference Sequence']
    elif len(locii) == 2 and detectable_muts:
        assert full_genome[locii[0]-1:locii[1]] == var_df.at[var_i, 'Reference Sequence']

var_df['var_locii'] = pd.Series(index=var_df.index, data=lociii)

var_df['Variant Location'] = var_df['Variant Location'].apply(lambda x: x.strip().upper())

var_df

,Variant Position,Variant Type,Reference Sequence,Variant Allele,Affected CG8421 Gene(s),CG8421 Genome Annotation,On/Off State In inoculum,On/Off State in Isolates,Variant Called in Inoculum or Isolate Population?,Isolate Population Samples That Differed From Inoculum,Relapse Samples with Variant (out of 6),Relapse Sample Frequency,Primary Infection Samples with Variant (out of 43),Primary Infection Sample Frequency,Annotation Used for this Study,Variant Location,var_locii
0,"67639,67640",Insertion,-,C,"CJ8421_RS00265, Promoter of CJ8421_RS00260",Hypothetical Protein,<10% Truncation,On,Inoculum,"Relapse 8.2 , Relapse 8.1 , Relapse 1 , Relaps...",4,66.666667,12,27.906977,fedA Hemerythrin,"IN GENE, IN PROMOTER REGION","(67639, 67640)"
1,"67639,67640",Insertion,-,CC,"CJ8421_RS00265, Promoter of CJ8421_RS00260",Hypothetical Protein,<10% Truncation,On,Inoculum,"Relapse 8.1 , Relapse 1 , Relapse 4 , Relapse ...",4,66.666667,43,100.000000,fedA Hemerythrin,"IN GENE, IN PROMOTER REGION","(67639, 67640)"
2,"100530,100531",Deletion,TA,-,CJ8421_RS00460,C4-dicarboxylate ABC transporter,On,Off,Infection Isolate(s),"16D5, 16D6",0,0.000000,2,4.651163,dcuA Anaerobic C4-dicarboxylate Transporter,IN GENE,"(100530, 100531)"
3,"168690,168691",Insertion,-,G,CJ8421_RS00870 & CJ8421_RS00875,hypothetical protein,On,<10% Truncation,Infection Isolate(s),Relapse 4,1,16.666667,0,0.000000,Conserved Hypothetical Proteins,"IN GENE, IN PROMOTER REGION","(168690, 168691)"
4,"168690,168691",Insertion,-,GG,CJ8421_RS00870,hypothetical protein,On,<10% Truncation,Infection Isolate(s),"Relapse 4 , Relapse 31",2,33.333333,0,0.000000,Conserved Hypothetical Proteins,IN GENE,"(168690, 168691)"
5,168691,Deletion,G,-,CJ8421_RS00870,hypothetical protein,On,<10% Truncation,Infection Isolate(s),"Relapse 8.2 , Relapse 9",2,33.333333,0,0.000000,Hypothetical Protein,IN GENE,"(168691,)"
6,268276,Deletion,T,-,CJ8421_RS01430,thiF family protein,On,Off,Infection Isolate(s),"Relapse 31 , 36D1, 8D3",1,16.666667,2,4.651163,thiF Family Protein,IN GENE,"(268276,)"
7,302194,SNV,A,G,CJ8421_RS01640,flagellar motor protein MotA,On,On,Infection Isolate(s),"11D5, 11D6",0,0.000000,2,4.651163,motA Falgellar Motor Protein,IN GENE,"(302194,)"
8,"335945,335946",Insertion,-,A,CJ8421_RS01805,TetR family transcriptional regulator,On,Off,Infection Isolate(s),Relapse 31,1,16.666667,0,0.000000,cmeR Bile Sensing Transcriptional Regulator,IN GENE,"(335945, 335946)"
9,335972,Deletion,T,-,CJ8421_RS01805,TetR family transcriptional regulator,On,Off,Infection Isolate(s),Relapse 4,1,16.666667,0,0.000000,cmeR Bile Sensing Transcriptional Regulator,IN GENE,"(335972,)"


#### The following code block performs the Fisher Exact test and bonferroni correction on the variations observed. Individually, very few rise to the 

In [5]:
from statsmodels.stats.multitest import multipletests
from scipy.stats import fisher_exact

var_file = '/Volumes/KeithSSD/variant-calling/misc_data/Crofts_Variants_Pooled.xlsx'
pooled_var_df = pd.read_excel(var_file, sheet_name='Sheet1', index_col=None)

col1 = 'Relapse Sample Count (out of 6)'
col2 = 'Relapse w/o Variant'
col3 = 'Primary Infection Samples with Variant(s) (out of 43)'
col4 = 'Primary Infections wo Variant'

pooled_var_df[col2] = 6 - pooled_var_df[col1]
pooled_var_df[col4] = 43 - pooled_var_df[col3]

oddsratio_fxn = lambda x: fisher_exact([[x[0], x[2]], [x[1], x[3]]])[0]
pvalue_fxn = lambda x: fisher_exact([[x[0], x[2]], [x[1], x[3]]])[1]

pooled_var_df['oddsratio'] = pooled_var_df[[col1, col2, col3, col4]].apply(oddsratio_fxn,  axis=1)
pooled_var_df['p_val'] = pooled_var_df[[col1, col2, col3, col4]].apply(pvalue_fxn,  axis=1)

corr_p_vals = multipletests(pooled_var_df['p_val'].values, method='bonferroni', alpha=0.05)
pooled_var_df['correct_p_val'] = pd.Series(corr_p_vals[1], index=pooled_var_df.index)
pooled_var_df["Relapse w/Variant"] = pooled_var_df[col1]
pooled_var_df["Primary Infection w/Variant"] = pooled_var_df[col3]

pooled_var_df['Reported Significant Associations'] = pooled_var_df['Variant Association with Human Disease State as Determined by Fisher Exact Test (FDR p-value <0.05)']

pooled_var_df[['Annotation', 'Reported Significant Associations', "Relapse w/Variant", col2, 
               "Primary Infection w/Variant", col4, 'p_val', 'oddsratio', 'correct_p_val']]

,Annotation,Reported Significant Associations,Relapse w/Variant,Relapse w/o Variant,Primary Infection w/Variant,Primary Infections wo Variant,p_val,oddsratio,correct_p_val
0,fedA Hemerythrin,NaN,6,0,43,0,1.000000,NaN,1.000000
1,dcuA Anaerobic C4-dicarboxylate Transporter,NaN,0,6,2,41,1.000000,0.000000,1.000000
2,Conserved Hypothetical Proteins,Relapse Isolates,4,2,0,43,0.000071,inf,0.002124
3,thiF Family Protein,NaN,1,5,2,41,0.330167,4.100000,1.000000
4,motA Falgellar Motor Protein,NaN,0,6,2,41,1.000000,0.000000,1.000000
5,cmeR Bile Sensing Transcriptional Regulator,Relapse Isolates,2,4,0,43,0.012755,inf,0.382653
6,Membrane GTP Binding Protein,NaN,1,5,2,41,0.330167,4.100000,1.000000
7,Flagella Modification Gene*,NaN,1,5,9,34,1.000000,0.755556,1.000000
8,Potassium Transporter KtrB; disrupted,NaN,2,4,3,40,0.106890,6.666667,1.000000
9,cipA Invasion Protein,Relapse Isolates,6,0,8,35,0.000215,inf,0.006442


#### I noticed that some of the locus tags listed in their Supplemental Data file have been replaced with new ones in the latest revision of the CG8421 genome, so this code goes through the variation dataframe and makes those same updates.

In [6]:
# fix formatting
var_df['affected'] = var_df["Affected CG8421 Gene(s)"].str.replace(", Promoter of ", " & ")
var_df['affected'] = var_df['affected'].apply(lambda x: x.split(" & "))

# manually verified = [12, 13, 14, 25, 26, 29, 30, 41, 42]
updated_tags = {"CJ8421_RS03020": "CJ8421_RS08905", 
                "CJ8421_RS03305": "CJ8421_RS08915", 
                "CJ8421_RS06480": "CJ8421_RS08860",
                'CJ8421_RS06580': 'CJ8421_RS08870',
                'CJ8421_RS06585': 'CJ8421_RS08870',
                "CJ8421_RS07090": "CJ8421_RS07085"}
def fix_tags(x):
    y = []
    for ip in x:
        i = ip.strip()
        if not i in updated_tags.keys():
            y.append(i)
        else:
            y.append(updated_tags[i])
    return list(set(y))

var_df['affected_2'] = var_df['affected'].apply(fix_tags)
changes_made = var_df[['affected', 'affected_2']].apply(lambda x: len(set(x[0]) - set(x[1])), axis=1).sum()
print("{} locus tags were either changed or removed".format(changes_made))


10 locus tags were either changed or removed


#### Here we will confirm the adjacency of each mutation and its affected gene. In the same script, we can add a column to indicate whether the mutation was detected within or between genes and another to collect the identifiers for additional nearby features that may add support for whether a paralog to the target region in the CG8421 genome is detected in our Peruvian isolates.

In [7]:
# all gene & cds are 1:1 in the records & location searches will be hampered by the source rec
ngb_df = gb_df.copy()[~gb_df['type'].isin(['gene', 'source'])]

var_cols = ['Variant Position', "Variant Type", "Reference Sequence", "Variant Allele"]

anchor_dicts, anch_counter = {}, 0 
for var_i in var_df.index:
    mut_pos = var_df.loc[var_i, 'var_locii'][0]
    mut_us, mut_ds = mut_pos + 150, mut_pos - 150
    coding_bool = ngb_df['type'] == 'CDS'
    within_bool = (ngb_df['start'] <= mut_pos) & (ngb_df['end'] >= mut_pos) & (coding_bool)
    downstream = (ngb_df['start'] > mut_pos) & (ngb_df['start'] <= mut_us) & (coding_bool)
    upstream = (ngb_df['end'] < mut_pos) & (ngb_df['end'] >= mut_ds) & (coding_bool)
    sensor_bools = [(downstream) & (ngb_df['strand'] == 1),
                    (upstream) & (ngb_df['strand'] == 1), 
                    (downstream) & (ngb_df['strand'] == -1),
                    (upstream) & (ngb_df['strand'] == -1), 
                    within_bool]
     
    sensor_types = ['upstream', 'downstream', 'upstream', 'downstream', 'in_CDS']
    keys_needed = ['locus_tag', 'start', 'end', 'strand', 'product', 'sequence', 'pseudo']
    all_pinged = within_bool.sum()+downstream.sum()+upstream.sum()
     
    print("{}. {} possible genes affected by mutation at {}".format(var_i, all_pinged, mut_pos))
    #print("\t= {}".format(var_df.loc[var_i, 'affected_2']))
    #print("{}, {}".format(mut_pos, ngb_df.loc[within_bool | upstream | downstream, keys_needed].T))
    #input()
     
    for sensor, senseType in zip(sensor_bools, sensor_types):
        if sensor.sum() > 0:
            sensee = ngb_df.loc[sensor, keys_needed]
            for hit_i in sensee.index:
                pos_dict = {'variation_index':var_i, 'mut_start':mut_pos, 'var_location': senseType}
                for vc in var_cols:
                    pos_dict[vc] = var_df.loc[var_i, vc]
                for kn in keys_needed:
                    pos_dict[kn] = sensee.loc[hit_i, kn]
                if pos_dict['strand'] == 1:
                    pos_dict['upstream_seq'] = full_genome[pos_dict['start']-20-1:pos_dict['start']+180]
                    pos_dict['downstream_seq'] = full_genome[pos_dict['end']-20-1:pos_dict['end']+180]
                    pos_dict['upstream_region'] = (pos_dict['start']-20, pos_dict['start']+180)
                    pos_dict['downstream_region'] = (pos_dict['end']-20, pos_dict['end']+180)
                else:
                    pos_dict['upstream_seq'] = full_genome[pos_dict['end']-20-1:pos_dict['end']+180]
                    pos_dict['downstream_seq'] = full_genome[pos_dict['start']-20-1:pos_dict['start']+180]
                    pos_dict['upstream_region'] = (pos_dict['end']-20, pos_dict['end']+180)
                    pos_dict['downstream_region'] = (pos_dict['start']-20, pos_dict['start']+180)
                anchor_dicts[anch_counter] = pos_dict
                anch_counter += 1

0. 2 possible genes affected by mutation at 67639
1. 2 possible genes affected by mutation at 67639
2. 1 possible genes affected by mutation at 100530
3. 2 possible genes affected by mutation at 168690
4. 2 possible genes affected by mutation at 168690
5. 2 possible genes affected by mutation at 168691
6. 1 possible genes affected by mutation at 268276
7. 1 possible genes affected by mutation at 302194
8. 1 possible genes affected by mutation at 335945
9. 1 possible genes affected by mutation at 335972
10. 2 possible genes affected by mutation at 517620
11. 2 possible genes affected by mutation at 517621
12. 1 possible genes affected by mutation at 565752
13. 1 possible genes affected by mutation at 565752
14. 1 possible genes affected by mutation at 620468
15. 1 possible genes affected by mutation at 629686
16. 1 possible genes affected by mutation at 629687
17. 2 possible genes affected by mutation at 680285
18. 1 possible genes affected by mutation at 686224
19. 1 possible genes aff

#### Here we are going to add some genes we are fans of and some random genes to search for just to get a sense of the range of matches observed in the data

#### To determine if our mutation is conserved we will look for the interrupted gene, as well as some nearby genes to determine how conserved the region is. To do this, we will extract the following anchor sequences per feature:
#### -  75 nucleotides on either side of mutation 
#### -  Gene sequences of 2 closest genes on either side and target gene
#### - Protein sequences for all gene anchors

In [8]:
conserved_ = ngb_df['locus_tag'].isin(['CJ8421_RS08380', 'CJ8421_RS00020', 'CJ8421_RS00430',
                                       'CJ8421_RS00435', 'CJ8421_RS02005', 'CJ8421_RS05885', 
                                       'CJ8421_RS06285', 'CJ8421_RS07345', 'CJ8421_RS07350', 
                                       'CJ8421_RS07355'])

random_recs = ngb_df[coding_bool].sample(n=10, random_state=1)
random_recs = random_recs.append(ngb_df[conserved_])

print("Adding {} genes to search for, currently there are: {}".format(len(random_recs), 
                                                                      len(anchor_dicts)))
for hit_i in random_recs.index:
    pos_dict = {}
    for vc in var_cols+['variation_index', 'mut_start', 'var_location']:
        pos_dict[vc] = np.nan
    for kn in keys_needed:
        pos_dict[kn] = ngb_df.loc[hit_i, kn]
    if pos_dict['strand'] == 1:
        pos_dict['upstream_seq'] = full_genome[pos_dict['start']-20-1:pos_dict['start']+180]
        pos_dict['downstream_seq'] = full_genome[pos_dict['end']-20-1:pos_dict['end']+180]
        pos_dict['upstream_region'] = (pos_dict['start']-20, pos_dict['start']+180)
        pos_dict['downstream_region'] = (pos_dict['end']-20, pos_dict['end']+180)
    else:
        pos_dict['upstream_seq'] = full_genome[pos_dict['end']-20-1:pos_dict['end']+180]
        pos_dict['downstream_seq'] = full_genome[pos_dict['start']-20-1:pos_dict['start']+180]
        pos_dict['upstream_region'] = (pos_dict['end']-20, pos_dict['end']+180)
        pos_dict['downstream_region'] = (pos_dict['start']-20, pos_dict['start']+180)
    anchor_dicts[anch_counter] = pos_dict
    anch_counter += 1

anchor_df = pd.DataFrame(anchor_dicts).T
print("Total of {} genes to discover i.e. {} primers".format(len(anchor_df), len(anchor_df)*3))
anchor_df = anchor_df.drop_duplicates()
print("Total of {} genes to discover i.e. {} primers".format(len(anchor_df), len(anchor_df)*3))
anchor_df.loc[[19, 20], :]

Adding 20 genes to search for, currently there are: 65
Total of 85 genes to discover i.e. 255 primers
Total of 85 genes to discover i.e. 255 primers


,Reference Sequence,Variant Allele,Variant Position,Variant Type,downstream_region,downstream_seq,end,locus_tag,mut_start,product,pseudo,sequence,start,strand,upstream_region,upstream_seq,var_location,variation_index
19,G,-,565752,Deletion,"(566403, 566603)",ATTTTAAATATAACAAATTGAGTATATTAAATGTTTATATTTAAAA...,566423,CJ8421_RS08905,565752,DUF2920 family protein,,ATGATTATAAACCAAATTTATTCTATAGATTCTTGTGATGATGTAG...,565192,1,"(565172, 565372)",ATTTCCAAAAAAAGGTTTAATATGATTATAAACCAAATTTATTCTA...,in_CDS,12
20,GG,-,"565752,565753",Deletion,"(566403, 566603)",ATTTTAAATATAACAAATTGAGTATATTAAATGTTTATATTTAAAA...,566423,CJ8421_RS08905,565752,DUF2920 family protein,,ATGATTATAAACCAAATTTATTCTATAGATTCTTGTGATGATGTAG...,565192,1,"(565172, 565372)",ATTTCCAAAAAAAGGTTTAATATGATTATAAACCAAATTTATTCTA...,in_CDS,13


#### Here we create a new folder for all data written to disk.
#### Then we write out both anchor files into files called reference_anchors.

In [9]:
anchor_df['before_header'] = anchor_df['locus_tag'].apply(lambda x: "{}-before".format(x))
anchor_df['after_header'] = anchor_df['locus_tag'].apply(lambda x: "{}-after".format(x))
anchor_df['gene_header'] = anchor_df['locus_tag'].apply(lambda x: "{}-gene".format(x))

def table_to_fasta(h_col, s_col, df, outname):
    output = list(df[[h_col, s_col]].apply(lambda x: ">"+str(x[0])+"\n"+x[1], axis=1))
    with open(outname, 'w') as fh:
        fh.write("\n".join(output)+"\n")
    print("\n", outname, "written")
    return

data_dir = "/Volumes/KeithSSD/variant-calling/variant_homologs"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

gene_file = os.path.join(data_dir, 'gene_primers.fa')

col_pairs = [('downstream_seq', 'before_header'), 
             ('upstream_seq', 'after_header'), 
             ('sequence', 'gene_header')]

outdf = pd.DataFrame(columns=['headers', 'nucleotides'], 
                     data=[list(x) for s, h in col_pairs for x in anchor_df[[h, s]].values]).drop_duplicates()

table_to_fasta('headers', 'nucleotides', outdf, gene_file)



 /Volumes/KeithSSD/variant-calling/variant_homologs/gene_primers.fa written


#### Here we will read in a text file containing information on the primary infection and relapse isolate pairs identified in both our and the Crofts et al. studies.  We use this information, and two columns from the Crofts supplement to define a new column, each row of which contains a list of the isolates expected to have a mutation in a given location 

In [10]:
metadata_file = "/Volumes/KeithSSD/CA_V3/data/10_functional_enrichment_final/enrichment_categories.txt"
metadata_df = pd.read_csv(metadata_file, sep="\t", index_col=0)
relapse_pair_isos = list(metadata_df[metadata_df['primary_v_relapse'].notnull()].index)
print("{} relapse pair strains identified, including:\n\t{}\n".format(len(relapse_pair_isos), relapse_pair_isos))
all_crofts_ = [i for i in metadata_df.index if 'crofts' in i]
relapse_pair_isos = list(set(relapse_pair_isos + all_crofts_))
print("At {} after adding the rest of Crofts isolates.".format(len(relapse_pair_isos)))

def fix_pop_col(x):
    f1 = lambda x: x.replace("Relapse 8.", "crofts_8R")
    f2 = lambda x: x.replace("Relapse 1", "crofts_1R")
    f3 = lambda x: x.replace("Relapse 9", "crofts_31R")
    f4 = lambda x: x.replace("Relapse 4", "crofts_9R")
    f5 = lambda x: x.replace("Relapse 31", "crofts_4R")
    f6 = lambda x: "crofts_" + x if 'D' in x else x
    f7 = lambda x: f6(f1(f2(f3(f4(f5(x.strip()))))))
    return list(map(f7, x.split(",")))

cn1 = 'Isolate Population Samples That Differed From Inoculum'
cn2 = 'Variant Called in Inoculum or Isolate Population?'

var_df['Isos_unlike_Innoc'] = var_df[cn1].apply(fix_pop_col)
f8 = lambda x: set(['crofts_innoc']) if x[0] == 'Inoculum' else set(x[1]).intersection(set(relapse_pair_isos))
f9 = lambda x: set(relapse_pair_isos).intersection(x[1]) if x[0] == 'Inoculum' else set()
var_df['Samps_w_Mutation'] = var_df[[cn2, 'Isos_unlike_Innoc']].apply(f8, axis=1)
var_df['Samps_off_Target'] = var_df[[cn2, 'Isos_unlike_Innoc']].apply(f9, axis=1)

for add_cols_ in ['Samps_w_Mutation', 'Samps_off_Target']:
    anchor_df[add_cols_] = anchor_df['variation_index'].map(var_df[add_cols_].to_dict())



24 relapse pair strains identified, including:
	['CAMPY82_spr11', 'pCAMPY80', 'CAMPY87_S44', 'crofts_8R2', 'crofts_4D1', 'CAMPY15_S7', 'crofts_8D3', 'crofts_4R', 'crofts_9D2', 'pCAMPY22', 'crofts_8R1', 'CAMPY4_spr11', 'pCAMPY47', 'CAMPY28', 'crofts_1R', 'crofts_31R', 'CAMPY12_S6', 'crofts_9R', 'CAMPY41_S20', 'CAMPY10', 'crofts_8D2', 'crofts_1D2', 'crofts_31D2', 'pCAMPY86']

At 34 after adding the rest of Crofts isolates.


#### We will now import full genome sequences and predicted protein sequences. The protein tables were exported from an'vio

In [11]:
prot_dir = "/Volumes/KeithSSD/CA_V3/data/03_CNTG_DBs_final"
prot_paths = [os.path.join(prot_dir, i+"_Proteins.txt") for i in relapse_pair_isos]
assert sum([os.path.exists(i) for i in prot_paths]) == len(prot_paths)
pos_paths = [os.path.join(prot_dir, i+"_GenePositions.txt") for i in relapse_pair_isos]
assert sum([os.path.exists(i) for i in pos_paths]) == len(pos_paths)
contigs_dir = "/Volumes/KeithSSD/CA_V3/data/02_ASSEMBLY_final"
assem_paths = [os.path.join(contigs_dir, i+"_contigs_fixed.fa") for i in relapse_pair_isos]
assert sum([os.path.exists(i) for i in assem_paths])

relapse_pair_isos.append('CG8421_sim')
pos_paths.append('/Volumes/KeithSSD/variant-calling/simulated_reference/CG8421_GenePositions.txt')
prot_paths.append('/Volumes/KeithSSD/variant-calling/simulated_reference/CG8421_Proteins.txt')
assem_paths.append('/Volumes/KeithSSD/variant-calling/simulated_reference/CG8421_sim_contigs_fixed.fa')

prot_dict = {}
print("Reading protein and gene position files")
for samp, prot_f, pos_f in zip(relapse_pair_isos, prot_paths, pos_paths):
    pos_df = pd.read_csv(pos_f, sep="\t", index_col=0)
    prot_df = pd.read_csv(prot_f, sep="\t", index_col=0)
    protpos_df = prot_df.join(pos_df, on=None, how='outer').reset_index()
    assert len(pos_df) == len(prot_df) == len(protpos_df)
    protpos_df['genome_name'] = pd.Series(index=protpos_df.index, 
                                       data=[samp]*len(protpos_df))
    prot_dict[samp] = protpos_df.copy()

def read_contig_file(contig_path, samp_name):
    header_lines, contig_sequences = [], []
    with open(contig_path, 'r') as fh:
        for line in fh:
            if line.startswith(">"):
                if len(header_lines) > 0:
                    contig_sequences.append(temp_sequence)
                header_lines.append(line[1:].strip())
                temp_sequence = ""
            else:
                temp_sequence += line.strip()
    
    contig_sequences.append(temp_sequence)
    if len(header_lines) != len(contig_sequences):
        print(len(header_lines), len(contig_sequences))
    assert len(header_lines) == len(contig_sequences)
    
    genome_size = len("".join(contig_sequences))
    new_headers = [samp_name + "-" + j + "-" + 'x' for i, j in enumerate(header_lines)]
    contig_df = pd.DataFrame(data=[header_lines, new_headers, contig_sequences], index=["old_ids", 'new_ids', 'sequence']).T
    return contig_df

print("Reading assemblies")
contigs = {i:read_contig_file(j, i) for i, j in zip(relapse_pair_isos, assem_paths)}

all_prot_df = pd.concat(prot_dict.values(), ignore_index=True, axis=0)
hdr_fxn = lambda x: "-".join([str(i) for i in x])
all_prot_df['new_ids'] = all_prot_df[['genome_name', 'contig', 'gene_callers_id']].apply(hdr_fxn, axis=1)
print("Total # proteins read {}".format(len(all_prot_df)))

all_cntg_df = pd.concat(contigs.values(), ignore_index=True, axis=0)
print("Total # contigs read {}".format(len(all_cntg_df)))

print(all_cntg_df.tail())
print(all_prot_df.tail())

gene_cluster_file = '../../CA_V3/data/07_PAN_GENOME_final/Trial_2/gene_clusters.txt'
cluster_df = pd.read_csv(gene_cluster_file, sep="\t")

cluster_df['mapper'] = cluster_df[['gene_caller_id', 'genome_name']].apply(tuple, axis=1)
clust_map = {k:v for k, v in list(cluster_df[['mapper', 'gene_cluster_id' ]].values)}

all_prot_df['mapper'] = cluster_df[['gene_caller_id', 'genome_name']].apply(tuple, axis=1)
all_prot_df['gene_cluster_id'] = all_prot_df['mapper'].map(clust_map)



Reading protein and gene position files
Reading assemblies
Total # proteins read 147064
Total # contigs read 34795
              old_ids                      new_ids  \
34790  c_000000000014  CG8421_sim-c_000000000014-x   
34791  c_000000000015  CG8421_sim-c_000000000015-x   
34792  c_000000000016  CG8421_sim-c_000000000016-x   
34793  c_000000000017  CG8421_sim-c_000000000017-x   
34794  c_000000000018  CG8421_sim-c_000000000018-x   

                                                sequence  
34790  AAATTTTCTGCTAATTTCTTATTTAACTCCAAAACATCTTCATGAA...  
34791  TCGTAGCTTTATTAGCATATTTTTTTAATAAAGGCAATTGTTTAGT...  
34792  TCGTAGCTTTATTAGCATATTTTTTTAATAAAGGCAATTGTTTAGT...  
34793  TGTTCTTCTAAACCTTTCTTTCAATTTTCTAGAATTTATTTTTTAG...  
34794  CATCCTAGTTATAAAGAATTACTTTCTAAAAGAAAGGGTATAAGTG...  
        gene_callers_id                                           sequence  \
147059             1716  MSHYYERFHEDVLELNKKLAENFKNSIVSHGNDPLDALQGIEQFVY...   
147060             1717             VALLAYFFNKGNCL

#### We will add the reference genome to these dataframes as a positive control and then we will print the correctly formatted commands to blast in the Terminal. We want to enforce 45% query coverage for both exact and full gene sequences. For the latter, it is because no matter where a frameshift occurs in a protein, at least 50% of one of the frames will still match the original. For the former, it is because the mutation is expected in the middle of the sequence and if the alignment decides to break, instead of pushing through the mismatch, a little under 50% of it should still be matched. 

In [12]:
src_index = list(gb_df[gb_df['type'] == 'source'].index)
assert len(src_index) == 1
header_pfx = "-".join(list(gb_df.loc[src_index[0], ['strain', 'molecule']]) + ['x'])
ref_row = pd.DataFrame({all_cntg_df.shape[0]:{'old_ids':"", 'new_ids':header_pfx, 'sequence': full_genome}}).T
all_cntg_df3 = all_cntg_df.append(ref_row, sort=True)
all_cntg_df2 = all_cntg_df3[['new_ids', 'sequence']].dropna()
print("Contigs number upped from {} to {}".format(len(all_cntg_df), len(all_cntg_df2)))

nt_ref_file = os.path.join(data_dir, 'contig_pile.fa')
ntdb = nt_ref_file.replace(".fa", "_db")
gene_out = os.path.join(data_dir, 'gene_hits.txt')
table_to_fasta('new_ids', 'sequence', all_cntg_df2, nt_ref_file)

print('\nOUTFMT="6 qseqid sseqid pident length qcovs mismatch gapopen qstart qend qlen sstart send slen evalue bitscore sstrand"\n')
print("makeblastdb -in {} -dbtype nucl -out {}\n".format(nt_ref_file, ntdb))
print('tblastx -query_gencode 11 -db_gencode 11 -evalue 1e-3 -db {} -query {} -outfmt "$OUTFMT" -qcov_hsp_perc 45 -out {} -num_threads 3 \n'.format(ntdb, gene_file, gene_out))


Contigs number upped from 34795 to 34796

 /Volumes/KeithSSD/variant-calling/variant_homologs/contig_pile.fa written

OUTFMT="6 qseqid sseqid pident length qcovs mismatch gapopen qstart qend qlen sstart send slen evalue bitscore sstrand"

makeblastdb -in /Volumes/KeithSSD/variant-calling/variant_homologs/contig_pile.fa -dbtype nucl -out /Volumes/KeithSSD/variant-calling/variant_homologs/contig_pile_db

tblastx -query_gencode 11 -db_gencode 11 -evalue 1e-3 -db /Volumes/KeithSSD/variant-calling/variant_homologs/contig_pile_db -query /Volumes/KeithSSD/variant-calling/variant_homologs/gene_primers.fa -outfmt "$OUTFMT" -qcov_hsp_perc 45 -out /Volumes/KeithSSD/variant-calling/variant_homologs/gene_hits.txt -num_threads 3 



#### After running BLAST, we read back in the results. BLAST can be tricky because if you allow for very lenient alignment parameters, the program will not find optimal alignments, however if you err on the side of conservatism, you risk false negatives. With that in mind, we made sure to use BLAST command parameters and post-processing filtering that allowed the program to get a minimum of 1 perfect match. We then set an identity limit by cutting and pasting the primers into the BLAST web search tool and performing tblasx against the RefSeq Campylobacter coli genome. The hits and misses were obvious and so we set an addtional, such that all substandard alignments were removed. 

In [13]:
from itertools import product

def best_pairs(df):
    primers_ = df['primer'].unique()
    genomes_ = df['genome_name'].unique()
    subdfs = [df[(df['primer'] == p) & (df['genome_name'] == g)] for p, g in product(primers_, genomes_)]
    subdfs2 = [df[df.evalue == df.evalue.min()] for df in subdfs]
    bestidxs = [i for df in subdfs2 for i in df[df.pident_adj == df.pident_adj.max()].index]
    df2 = df.loc[bestidxs, :].copy()
    df3 = df2[df2.pident > 80.0 ]
    check_ = [df3[(df3['primer'] == p) & (df3['genome_name'] == 'CG8421')].shape[0] > 0 for p in primers_]
    assert set(check_) == set([True])
    return df3.copy()

blast_cols = ['primer', 'sseqid', 'pident', 'length', 'qcov', 'mismatch', 'gapopen', 'qstart', 'qend',
              'qlen', 'sstart', 'send', 'slen', 'evalue', 'bitscore', 'sstrand']
new_cols = ['genome_name', 'contig', 'gene_callers_id', 'pident_adj', 'evalue']
extra_cols = set(blast_cols[5:]) - set(new_cols)
new_order = blast_cols[:5] + new_cols + list(extra_cols) + ['present', 'sstart2', 'send2']

hit_df = pd.read_csv(gene_out, sep="\t", header=None)
hit_df.columns = blast_cols
hit_df['genome_name'] = hit_df['sseqid'].apply(lambda x: x.split("-")[0])
hit_df['contig'] = hit_df['sseqid'].apply(lambda x: x.split("-")[1])
hit_df['gene_callers_id'] = hit_df['sseqid'].apply(lambda x: x.split("-")[2])
hit_df['qcov'] = abs(hit_df['qend'] - hit_df['qstart']) / hit_df['qlen']
hit_df['sstart2'] = hit_df[['sstart', 'send']].apply(lambda x: min(x), axis=1)
hit_df['send2'] = hit_df[['sstart', 'send']].apply(lambda x: max(x), axis=1)
hit_df['pident_adj'] = hit_df['pident']*hit_df['qcov']
hit_df['present'] = pd.Series(index=hit_df.index, data=[1]*len(hit_df))
print("Raw BLAST hits total {} w/ {} columns".format(len(hit_df), hit_df.shape[1]))
assert set(new_order) == set(hit_df.columns)
blast_df = best_pairs(hit_df[new_order])
blast_df['locus_tag'] = blast_df['primer'].apply(lambda x: x.split("-")[0])
blast_df['primer_position'] = blast_df['primer'].apply(lambda x: x.split("-")[1])
print("\tAfter keeping only best pairs, we are left with {}".format(blast_df.shape[0]))

Raw BLAST hits total 33823 w/ 23 columns
	After keeping only best pairs, we are left with 6690


#### Here we compile both types of BLAST searches and pivot them into a presence/absence table with primers in the columns and genomes in the rows. We will also make a similarly shaped table with all the location information for each hit.

In [14]:
crofts_genomes = list(filter(lambda x: x.startswith("crofts"), blast_df['genome_name'].unique()))
my_genomes = [i for i in set(blast_df['genome_name']) if not i.startswith("CG8421") and not i in crofts_genomes]

def fix_aggs(x):
    x.index.name = None
    x.columns = x.columns.droplevel()
    x.columns.name = None
    return x

# Create a location to record the # of valid homologs per primer
pre_homolog = blast_df[['genome_name', 'primer', 'present']].groupby(['genome_name', 'primer'])
homolog_df = fix_aggs(pre_homolog.agg(np.sum).unstack(level=-1))

# Create permanent + temporary locations to record the corresponding contig and location values 
loc_cols = ['sseqid', 'sstart2', 'send2', 'qstart', 'qend', 'pident']
blast_df['loc_info'] = blast_df[loc_cols].apply(tuple, axis=1)
pre_loc = blast_df[['genome_name', 'primer', 'loc_info']].groupby(['genome_name', 'primer'])
location_df = fix_aggs(pre_loc.agg(list).unstack(level=-1))

## Create a location for e-values and set permissive defaults 
pre_eval = blast_df[['genome_name', 'primer', 'evalue']].groupby(['genome_name', 'primer'])
evalue_df = fix_aggs(pre_eval.agg(list).unstack(level=-1))

searched_for = homolog_df.shape[0]*homolog_df.shape[1]
detected_n = searched_for - homolog_df.isnull().sum().sum()
print("Total possible homologs = {}, Total valid ones detected = {}".format(searched_for, detected_n))

coldOrder = ['CG8421', 'CG8421_sim'] + crofts_genomes + my_genomes
assert set(coldOrder) - set(evalue_df.index) == set()

homolog_group_order = []
for aff_gene in anchor_df['locus_tag'].unique():
    homolog_group_order += [aff_gene+'-before', aff_gene+'-gene', aff_gene+'-after']

assert set(homolog_group_order) == set(homolog_df.columns)
homolog_df.loc[coldOrder, homolog_group_order].to_csv("/Volumes/KeithSSD/variant-calling/misc_data/homologs.txt", sep="\t")
evalue_df.loc[coldOrder, homolog_group_order].to_csv("/Volumes/KeithSSD/variant-calling/misc_data/evalues.txt", sep="\t")
location_df.loc[coldOrder, homolog_group_order].to_csv("/Volumes/KeithSSD/variant-calling/misc_data/locations.txt", sep="\t")


Total possible homologs = 6372, Total valid ones detected = 5707


#### Now that we know where the hits are, we need to integrate them with the variation dataframe file. We go through our BLAST hits by variant region and check that either the full gene or the exact region in which a variation was detected was on the same contig as at least one of the genes affected by that variation. Sometimes the variation is within the gene, and so this is a redundant check, but many times they are not. We also go through and check to see if BLAST was able to locate an exact copy of the wild-type allele (found in the reference genome) in the isolate assembly. 

In [24]:
def find_wt(gnsr, ra):
    cntg_df, gnc = all_cntg_df3.copy(), list(gnsr)
    if type(ra) != float:
        cntg_match = cntg_df['new_ids'] == gnc[0]
        if cntg_match.sum() == 1:
            cntg_seq = cntg_df.loc[cntg_match, 'sequence'].values[0]
            for sub_seq in ra:
                if cntg_seq.find(sub_seq, gnc[1]-20, gnc[2]+20) != -1:
                    return True
            return False
        else:
            raise ValueError("what contig from where: {}".format(gnc))
    else:
        return ra

from Bio.Seq import Seq
from Bio.Alphabet import generic_dna

def seq_array(ra):
    ra2 = Seq(ra, generic_dna)
    ra_ls = [ra]
    ra_ls.append(str(ra2.reverse_complement()))
    ra_ls.append(str(ra2.complement()))
    ra_ls.append(str(ra2.reverse_complement().complement()))
    return ra_ls

## Repeat the iteration from before 
data_prod_1 = []

pop_cols = ['has_wildtype', 'has_homolog']
for pop_colx in pop_cols:
    anchor_df[pop_colx] = pd.Series(index=anchor_df.index) 

contigs_identified = set()
loc_cols2 = ['sseqid', 'sstart2', 'send2']

for ix in anchor_df.index:
    gene_, _var_ = list(anchor_df.loc[ix, ['locus_tag', 'variation_index']])
    if type(_var_) != float:
        locii, annot_ = list(var_df.loc[_var_, ['var_locii', 'Annotation Used for this Study']])
        start_cut = locii[0]-1-8
        if len(locii) == 1:
            end_cut = locii[0]+8
        else:
            end_cut = locii[1]+8
        ref_set = seq_array(full_genome[start_cut:end_cut][:16])
    else:
        ref_set = np.nan
        annot_ = ngb_df.loc[ngb_df['locus_tag'] == gene_, 'product'].values[0]
        locii = (0,0)
    
    blast_filt = blast_df.copy()[blast_df['locus_tag'] == gene_]
    
    # pull out populations with the homologous region
    reg_by_iso = blast_filt[['genome_name', 'primer_position']].groupby('genome_name').agg(lambda x: len(set(x)))
    good_isos = list(reg_by_iso[reg_by_iso['primer_position'] > 1 ].index)
    anchor_df.loc[ix, 'has_homolog'] = "@#$".join(good_isos)
    
    # search for the wild type allele 
    blast_filt['has_wildtype'] = blast_filt[loc_cols2].apply(lambda x: find_wt(x, ref_set), axis=1)
    wt_havers = set(blast_filt.loc[blast_filt['has_wildtype'] == True, 'genome_name'])
    anchor_df.loc[ix, 'has_wildtype'] = "@#$".join(list(wt_havers))

    # aggregate all the contigs that were identified 
    contigs_identified.update(list(blast_filt['sseqid'].unique()))
    print("{}\tGene = {}\t Samples w/it = {}\t Samples w/WT seq = {}\t{}".format(annot_, gene_, len(good_isos), 
                                                                             len(wt_havers), locii))
for pop_colx in pop_cols:
    nn_x = anchor_df[pop_colx].notnull()
    anchor_df.loc[nn_x, pop_colx] = anchor_df.loc[nn_x, pop_colx].apply(lambda x: x.split("@#$"))


In [25]:
#gene_ = 'CJ8421_RS06660'
#blast_filt = blast_df[blast_df['locus_tag'] == gene_]
#blast_filt.sample(1).T
anchor_df[pop_cols].head(2).values

array([[list(['CAMPY87_S44', 'crofts_24D2', 'crofts_31R', 'CAMPY15_S7', 'CAMPY12_S6', 'crofts_9R', 'pCAMPY22', 'crofts_32D2', 'CAMPY41_S20', 'crofts_4D1', 'crofts_11D1', 'CAMPY82_spr11', 'crofts_8R1', 'crofts_14D2', 'crofts_21D1', 'crofts_16D0', 'CG8421_sim', 'crofts_31D2', 'pCAMPY47', 'crofts_4R', 'CG8421', 'pCAMPY80', 'crofts_8D3', 'crofts_8D2', 'crofts_14D3', 'crofts_9D2', 'crofts_innoc', 'crofts_16D1', 'crofts_8R2', 'crofts_33D1', 'CAMPY4_spr11', 'crofts_1R', 'crofts_1D2']),
        list(['CAMPY10', 'CAMPY12_S6', 'CAMPY15_S7', 'CAMPY28', 'CAMPY41_S20', 'CAMPY4_spr11', 'CAMPY82_spr11', 'CAMPY87_S44', 'CG8421', 'CG8421_sim', 'crofts_11D1', 'crofts_14D2', 'crofts_14D3', 'crofts_16D0', 'crofts_16D1', 'crofts_1D2', 'crofts_1R', 'crofts_21D1', 'crofts_24D2', 'crofts_31D2', 'crofts_31R', 'crofts_32D2', 'crofts_33D1', 'crofts_4D1', 'crofts_4R', 'crofts_8D2', 'crofts_8D3', 'crofts_8R1', 'crofts_8R2', 'crofts_9D2', 'crofts_9R', 'crofts_innoc', 'pCAMPY22', 'pCAMPY47', 'pCAMPY80'])],
       [l

#### Here we print a summary table where we look at regions of variation how often each region of variation is found in in primary and relapse isolates from both groups, as well as in the complete and reassembled versions of the reference. 

In [28]:
innoc = 'crofts_innoc'
ref_complete = 'CG8421'
ref_sim = 'CG8421_sim'
single_isos = ['Ref_Full', 'Ref_Sim', 'Crofts_Innoc'] 
single_keys = [ref_complete, ref_sim, innoc]

relapse_crofts = list(metadata_df[metadata_df['primary_v_relapse_crofts'] == 'Relapse'].index)
primary_crofts = [i for i in crofts_genomes if not i in single_keys and not i in relapse_crofts]
print(len(crofts_genomes), len(primary_crofts), len(relapse_crofts), len(single_keys))

peru_primary = list(metadata_df[metadata_df['primary_v_relapse_nocrofts'] == 'Primary'].index)
peru_relapse = list(metadata_df[metadata_df['primary_v_relapse_nocrofts'] == 'Relapse'].index)

group_names = ['Crofts_Primary', 'Crofts_Relapse', 'Iquitos_Primary', 'Iquitos_Relapse']
group_keys = [primary_crofts, relapse_crofts, peru_primary, peru_relapse]

procd_homologs = {}
for anch_idx in anchor_df.index:
    wt_isos, homolog_isos = anchor_df.loc[anch_idx, pop_cols]
    assert type(wt_isos) == list == type(homolog_isos)
    wt_isos, homolog_isos = set(wt_isos), set(homolog_isos)
    rec_rows = {}
    for g_n, g_k in zip(group_names, group_keys):
        g_ks = set(g_k)
        rec_rows["_".join([g_n, 'w_Wildtype'])] = len(wt_isos.intersection(g_ks))
        rec_rows["_".join([g_n, 'wo_Wildtype'])] = len(g_ks) - len(wt_isos.intersection(g_ks))
        rec_rows["_".join([g_n, 'w_Region'])] = len(homolog_isos.intersection(g_ks))
        rec_rows["_".join([g_n, 'wo_Region'])] = len(g_ks) - len(homolog_isos.intersection(g_ks))
        
    for s_i, s_k in zip(single_isos, single_keys):
        rec_rows["_".join([s_i, 'has_Wildtype'])] = s_k in wt_isos
        rec_rows["_".join([s_i, 'Region_Count'])] = s_k in homolog_isos

    procd_homologs[anch_idx] = rec_rows

# Need to map the descriptive information from the original dataframe to here 
flipped_df = pd.DataFrame(procd_homologs).T
e_anchor_df = pd.concat([anchor_df, flipped_df], axis=1)
print(flipped_df.shape, anchor_df.shape, e_anchor_df.shape)

flipped_df.to_csv('/Volumes/KeithSSD/variant-calling/misc_data/all_iso_all_region_processed.txt', sep="\t")

22 15 6 3
(85, 22) (85, 25) (85, 47)


#### Next we decide which of the two assemblies (primary or relapse) is more complete and then output a pair of columns in a text file of which assembly to use as a reference and which to library to map to it, in order to call variations. For the Croft's genomes, we will use the simulated and complete reference genomes, in order to reproduce their results.  

In [34]:
# find Iquitos relapse pairings from somewhere 
relpase_pair_file = '/Volumes/KeithSSD/CA_V3/data/09_Phylogenomics/relapse_dist_pairs_120719.txt'
iquitos_pairings = pd.read_csv(relpase_pair_file, sep="\t", index_col=0)[['Pair Member 1', 'Pair Member 2']]
all_genomes = set(blast_df['genome_name'])
ip_links = {i:j for i, j in iquitos_pairings.values 
                if (i in all_genomes) and (j in all_genomes) and (not i in crofts_genomes)}

# read in each desired contig set from each member of a pair
print("{} contigs hit across all genomes".format(len(contigs_identified)))
assert len(set(contigs_identified) - set(all_cntg_df2['new_ids'])) == 0

all_cntg_df2['source_genome'] = all_cntg_df2['new_ids'].apply(lambda x: x.split("-")[0])
all_cntg_df4 = all_cntg_df2.copy()[all_cntg_df2['new_ids'].isin(contigs_identified)]
all_cntg_df4['seqlen'] = all_cntg_df4['sequence'].apply(len)

contig_sum = all_cntg_df4[['source_genome', 'seqlen']].groupby('source_genome').agg(np.sum)['seqlen']

assem_lib_pairs = []
for cg in crofts_genomes:
    assem_lib_pairs.append(('CG8421', cg))

# whichever has a greater number of hits write out as the "backbone"
peru_isos = set(peru_primary + peru_relapse)
for k, v in ip_links.items():
    peru_isos.remove(k); peru_isos.remove(v);
    ixs = np.array([k, v])
    anylz = np.array([contig_sum[k], contig_sum[v]])
    assem_lib_pairs.append((ixs[anylz.argmax()], ixs[anylz.argmin()]))

# create a table of the name of the backbone and the name of the reads to map against it
mapping_pairs = pd.DataFrame(index=range(len(assem_lib_pairs)), data=assem_lib_pairs,
                             columns=['reference_assembly', 'mapping_query'])

# write a `snipper` file to do the mapping but alter it to find my contigs too
mapping_pairs.to_csv("/Volumes/KeithSSD/variant-calling/misc_data/ref_map_pairs.txt", sep="\t", 
                     header=None, index=None)
mapping_pairs

966 contigs hit across all genomes


,reference_assembly,mapping_query
0,CG8421,crofts_9D2
1,CG8421,crofts_4D1
2,CG8421,crofts_31D2
3,CG8421,crofts_11D1
4,CG8421,crofts_9R
5,CG8421,crofts_14D2
6,CG8421,crofts_8D2
7,CG8421,crofts_innoc
8,CG8421,crofts_4R
9,CG8421,crofts_16D0


In [35]:
# write contig sets from reference genomes into individual files in a directory 
ref_dir2 = "/Volumes/KeithSSD/variant-calling/ref_fastas"
if not os.path.exists(ref_dir2):
    os.mkdir(ref_dir2)

ref_set = list(mapping_pairs['reference_assembly'].unique())
out_paths = [os.path.join(ref_dir2, sn+"_bb.fa") for sn in ref_set]
sub_dfs = [all_cntg_df4.copy()[all_cntg_df4['source_genome'] == sn] for sn in ref_set]
for sn, of, sdf in zip(ref_set, out_paths, sub_dfs):
    print("Writing {} recs for {} to {}".format(len(sdf), sn, of))
    table_to_fasta('new_ids', 'sequence', sdf, of)


Writing 1 recs for CG8421 to /Volumes/KeithSSD/variant-calling/ref_fastas/CG8421_bb.fa

 /Volumes/KeithSSD/variant-calling/ref_fastas/CG8421_bb.fa written
Writing 29 recs for CAMPY28 to /Volumes/KeithSSD/variant-calling/ref_fastas/CAMPY28_bb.fa

 /Volumes/KeithSSD/variant-calling/ref_fastas/CAMPY28_bb.fa written
Writing 48 recs for pCAMPY80 to /Volumes/KeithSSD/variant-calling/ref_fastas/pCAMPY80_bb.fa

 /Volumes/KeithSSD/variant-calling/ref_fastas/pCAMPY80_bb.fa written
Writing 49 recs for CAMPY4_spr11 to /Volumes/KeithSSD/variant-calling/ref_fastas/CAMPY4_spr11_bb.fa

 /Volumes/KeithSSD/variant-calling/ref_fastas/CAMPY4_spr11_bb.fa written
Writing 46 recs for CAMPY87_S44 to /Volumes/KeithSSD/variant-calling/ref_fastas/CAMPY87_S44_bb.fa

 /Volumes/KeithSSD/variant-calling/ref_fastas/CAMPY87_S44_bb.fa written
Writing 45 recs for CAMPY41_S20 to /Volumes/KeithSSD/variant-calling/ref_fastas/CAMPY41_S20_bb.fa

 /Volumes/KeithSSD/variant-calling/ref_fastas/CAMPY41_S20_bb.fa written
Writing 

Once we've run prodigal to convert the FASTA files to genbank files and then Snippy to call variations between the reference backbone files and the libraries. We can proceed to read back the variants that were discovered.

In [38]:
# read in the SNP tables
snp_result_dir = "/Volumes/KeithSSD/variant-calling/snipout3_results"
snp_files = os.listdir(snp_result_dir)
snp_paths = [os.path.join(snp_result_dir, f) for f in snp_files if f.endswith(".2.tab")]
print("{} polymorphism reports detected".format(len(snp_paths)))
mp2 = [os.path.basename(i).replace(".2.tab", "").split("-") + [i] for i in snp_paths]
mapping_pairs2 = pd.DataFrame(data=mp2, columns=['ix', 'ref_assem', 'mapped_lib', 'result']).drop(['ix'], axis=1)
results_df = {i:pd.read_csv(mapping_pairs2.loc[i, 'result'], sep="\t") for i in mapping_pairs2.index}

bad_maps = (mapping_pairs2['ref_assem'] == 'CG8421') & (~mapping_pairs2['mapped_lib'].str.contains("crofts"))
bad_map_indexes = list(mapping_pairs2[bad_maps].index)
bad_map_variants = np.array([results_df[i].shape[0] for i in bad_map_indexes])

better_maps = (mapping_pairs2['ref_assem'] != 'CG8421') & (~mapping_pairs2['mapped_lib'].str.contains("crofts"))
better_map_indexes = list(mapping_pairs2[better_maps].index)
better_map_variants = np.array([results_df[i].shape[0] for i in better_map_indexes])

good_maps = (mapping_pairs2['ref_assem'] == 'CG8421') & (mapping_pairs2['mapped_lib'].str.contains("crofts"))
good_map_indexes = list(mapping_pairs2[good_maps].index)
good_map_variants = np.array([results_df[i].shape[0] for i in good_map_indexes])

best_maps = (mapping_pairs2['ref_assem'] == 'CG8421_sim') & (mapping_pairs2['mapped_lib'].str.contains("crofts"))
best_map_indexes = list(mapping_pairs2[best_maps].index)
best_map_variants = np.array([results_df[i].shape[0] for i in best_map_indexes])

print("Peru libs have a mean={:.2f}, std={:.2f} variations v. CG8421 ref (n={})".format(bad_map_variants.mean(),
                                                                                        bad_map_variants.std(),
                                                                                        bad_map_variants.shape[0]))

print("But a mean={:.2f}, std={:.2f} variations v. their own assems (n={})".format(better_map_variants.mean(),
                                                                                   better_map_variants.std(),
                                                                                   better_map_variants.shape[0]))

print("Crofts libs have a mean={:.2f}, std={:.2f} variations v. CG8421 ref (n={})".format(good_map_variants.mean(),
                                                                                          good_map_variants.std(),
                                                                                          good_map_variants.shape[0]))

print("and a mean={:.2f}, std={:.2f} variations v. CG8421 assem (n={})".format(best_map_variants.mean(),
                                                                               best_map_variants.std(),
                                                                               best_map_variants.shape[0]))

results_df = {k:v for k, v in results_df.items() if not k in bad_map_indexes}
print("{} SNP reports retained".format(len(results_df)))
#assert set(mapping_pairs['reference_assembly']) == set(mapping_pairs2['ref_assem'])
#assert set(mapping_pairs['mapping_query']) == set(mapping_pairs2['mapped_lib'])

62 polymorphism reports detected
Peru libs have a mean=11704.17, std=3952.42 variations v. CG8421 ref (n=6)
But a mean=6.50, std=11.33 variations v. their own assems (n=12)
Crofts libs have a mean=19.55, std=5.10 variations v. CG8421 ref (n=22)
and a mean=18.91, std=5.10 variations v. CG8421 assem (n=22)
56 SNP reports retained


In [48]:
print(e_anchor_df['Variant Type'].unique())
print(set([i for v in results_df.values() for i in v['TYPE'].unique()]))

print(e_anchor_df['var_type'].unique())
#list(results_df.values())[0]
#e_anchor_df.iloc[0, :].T

['Insertion' 'Deletion' 'SNV' nan]
{'complex', 'del', 'snp', 'mnp', 'ins'}
['ins' 'del' 'snp' nan]


In [55]:
# clean up the blast hits 
select_blast = blast_df[blast_df['sseqid'].isin(contigs_identified)]

# make variant types match 
e_anchor_df['var_type'] = e_anchor_df['Variant Type'].map({'Insertion':'ins', 'Deletion':'del', 'SNV':'snp'})

def calc_evidence(x):
    res_alt, res_evid = x
    print(res_alt, res_evid)
    evid_nums = [int(i.split(":")[-1].strip()) for i in res_evid.split()]
    assert res_alt == res_evid.split(":")[0].strip()
    return float(evid_nums[0])/float(sum(evid_nums))

# make one result df
res_pile = []
for a_res, res_dfx in results_df.items():
    ref, query = list(mapping_pairs2.loc[a_res, ['ref_assem', 'mapped_lib']])
    res_df = res_dfx.copy()
    res_df['ref_assem'] = pd.Series(index=res_df.index, data=[ref]*len(res_df))
    res_df['mapped_lib'] = pd.Series(index=res_df.index, data=[query]*len(res_df))
    res_pile.append(res_df)

all_results = pd.concat(res_pile, axis=0, ignore_index=True, verify_integrity=True)
all_results['evidence_pct'] = all_results[['ALT', 'EVIDENCE']].apply(calc_evidence, axis=1)
print(all_results.shape)
print(all_results.head())

# determine whether it matches the location and type of a recorded mutation 
for anch_idx in e_anchor_df.index:
    all_results['mut_start'] 
    assert False
    
for res_row in res_df.index:
    res_cntg, res_pos = res_df.loc[res_row, ['CHROM', 'POS']]
    q1 = select_blast['sseqid'] == res_cntg
    q2 = select_blast['sstart2'] <= res_pos
    q3 = select_blast['send2'] >= res_pos

    subhit_df = select_blast[((q1) & (q2) & (q3))]
    if (subhit_df.shape[0] > 0) and (evidence_pct > 0.25) and ref == 'CG8421':

        


        # confirm whether its upstream, downstream, or in CDS based on primer name 

        # 

        # 
        print(subhit_df.T)
        assert False
        res_df.loc[res_row, 'primer_match'] = ", ".join(list(subhit_df['primer']))
        l_cols2 = ['primer', 'send', 'sstart', 'qstart', 'qend', 'qlen']

    # add 
    if ref == 'CG8421':
        possible_affected = []
        # if its within a CDS
        within = (ngb_df['start'] <= res_pos) & (ngb_df['end'] >= res_pos)

        # closest downstream watson strand encoded gene 
        downstream = (ngb_df['strand'] == 1) & ((ngb_df['start'] - res_pos) > 0)
        if downstream.sum() > 0:
            min_start = (ngb_df.loc[downstream, 'start'] - res_pos).astype(int).idxmin()
            dist_to_start = ngb_df.loc[min_start, 'start'] - res_pos
            res_df.loc[res_row, 'nearest_downstream'] = ngb_df.loc[min_start, 'locus_tag']
            res_df.loc[res_row, 'dist_downstream'] = dist_to_start
            if abs(dist_to_start) <= 50:
                 possible_affected.append(res_df.loc[res_row, 'nearest_downstream'])

        # closest upstream crick strand encoded gene
        upstream = (ngb_df['strand'] == -1) & ((res_pos - ngb_df['end']) > 0)
        if upstream.sum() > 0:
            min_end = (res_pos - ngb_df.loc[upstream, 'end']).astype(int).idxmin()
            dist_to_end = res_pos - ngb_df.loc[min_start, 'end']
            res_df.loc[res_row, 'nearest_upstream'] = ngb_df.loc[min_end, 'locus_tag']
            res_df.loc[res_row, 'dist_upstream'] = dist_to_end
            if abs(dist_to_end) <= 50:
                 possible_affected.append(res_df.loc[res_row, 'nearest_upstream'])

        if within.sum() > 0:
            possible_affected += list(ngb_df.loc[within, 'locus_tag'])
            res_df.loc[res_row, 'affected_genes'] = ", ".join(possible_affected)
            res_df.loc[res_row, 'within_gene'] = ", ".join(list(ngb_df.loc[within, 'locus_tag']))
            #print("\t", res_row, res_cntg, res_pos, res_df.loc[res_row, 'within_gene'], list(ngb_df.loc[within, ['start', 'end', 'gene', 'product']].values))

        aff_gene_cntr.update(possible_affected)

results_df2[a_res] = res_df.copy()
print("mapping {} to {}, gives {} matched variants {} affected genes".format(ref, query, hit_counter, 
                                                                             len(aff_gene_cntr)))
print("\t{}\n".format(aff_gene_cntr))
          

drp_cols = ['AA_POS', 'EFFECT', 'FTYPE', 'GENE', 'LOCUS_TAG', 'NT_POS', 'PRODUCT', 'STRAND']
snp_df = pd.concat(list(results_df2.values()), axis=0, ignore_index=True, sort=True).drop(drp_cols, axis=1)
good_snps_df = snp_df.copy()[snp_df['primer_match'].notnull()]
good_snps_df['primer_set'] = good_snps_df['primer_match'].apply(lambda x: set(x.split(", ")))
good_snps_df.sample(10)


GCATCATCCTCCTTTATTT GCATCATCCTCCTTTATTT:34 G:2
A A:148 AG:10
CTGAAAG CTGAAAG:42 ATAAAAA:0
CT CT:40 AA:0
AATAGA AATAGA:6 GTTATG:0
T T:100 TC:38
TACTAGTTA TACTAGTTA:72 GAATACTTG:4
C C:106 T:4
T T:116 C:4
AAAGCTGGAGTG AAAGCTGGAGTG:82 GAAAGTAGGAGTA:4
TT TT:164 AA:12
A A:156 AAG:0
A A:148 AC:14
A A:162 AG:12
T T:172 TA:0
CG CG:122 C:52
G G:112 A:0
AC AC:160 A:112
AC AC:154 A:40
A A:146 AC:14
A A:142 AC:12
A A:90 T:0
G G:70 C:0
GA GA:34 AT:0
CCT CCT:32 C:0
A A:10 T:2
GAA GAA:10 AT:0
TACTAGTTA TACTAGTTA:72 GAATACTTG:4
C C:106 T:4
T T:116 C:4
AAAGCTGGAGTG AAAGCTGGAGTG:82 GAAAGTAGGAGTA:4
TT TT:164 AA:12
A A:156 AAG:0
A A:148 AC:14
A A:162 AG:12
T T:172 TA:0
CG CG:122 C:52
A A:146 AG:10
CTGAAAG CTGAAAG:42 ATAAAAA:0
CT CT:40 AA:0
AATAGA AATAGA:6 GTTATG:0
T T:146 TC:42
TTC TTC:10 AT:0
T T:10 A:2
AGGAAAATCATAACAATAAATT AGGAAAATCATAACAATAAATT:32 GAAAAATATAAGAATAAATA:0
CGATGAAATAAAGGAGGAT CGATGAAATAAAGGAGGAT:34 C:0
AG AG:154 A:48
TG TG:164 T:122
C C:112 T:0
G G:8 T:0
C C:132 G:0
T T:164 TG:16
T T:156

C C:112 G:0
T T:30 C:28
A A:15 G:0
C C:14 G:0
G G:6 A:6
A A:10 T:0
A A:12 C:1
C C:6 A:0
C C:11 T:5
T T:6 A:3
(924, 17)
                  CHROM     POS     TYPE      REF                  ALT  \
0  CG8421-NZ_CP005388-x  149010      ins        G  GCATCATCCTCCTTTATTT   
1  CG8421-NZ_CP005388-x  565751      del       AG                    A   
2  CG8421-NZ_CP005388-x  649500  complex  ATAAAAA              CTGAAAG   
3  CG8421-NZ_CP005388-x  649511  complex       AA                   CT   
4  CG8421-NZ_CP005388-x  649530  complex   GTTATG               AATAGA   

                     EVIDENCE FTYPE STRAND NT_POS AA_POS EFFECT LOCUS_TAG  \
0  GCATCATCCTCCTTTATTT:34 G:2   NaN    NaN    NaN    NaN    NaN       NaN   
1                 A:148 AG:10   NaN    NaN    NaN    NaN    NaN       NaN   
2        CTGAAAG:42 ATAAAAA:0   NaN    NaN    NaN    NaN    NaN       NaN   
3                  CT:40 AA:0   NaN    NaN    NaN    NaN    NaN       NaN   
4           AATAGA:6 GTTATG:0   NaN    NaN    NaN  

AssertionError: 

In [ ]:
substandard_repeats = []
for primer in sim_check_df.primer.unique():
    matched_locations = sim_check_df[sim_check_df['primer'] == primer]
    pident_adj = matched_locations['pident'] * ((matched_locations['qend'] - matched_locations['qstart'])/matched_locations['qend'])
    best_locations = matched_locations[pident_adj == pident_adj.max()]
    substandard_repeats += list(set(matched_locations.index) - set(best_locations.index))

sim_check_df2 = sim_check_df.drop(substandard_repeats, axis=0)

def convert_sim_locs(x):
    sscdf = sim_check_df2[sim_check_df2['primer'] == x[0]]
    if sscdf.shape[0] > 1:
        return np.nan
    elif sscdf.shape[0] == 1:
        assert sscdf['qstart'].values[0] < sscdf['qend'].values[0]
        if sscdf['sstart'].values[0] > sscdf['send'].values[0]:
            return sscdf['sstart'].values[0]  - (x[1] - sscdf['qstart'].values[0])
        else:
            return sscdf['sstart'].values[0]  + (x[1] - sscdf['qstart'].values[0])
    else:
        raise ValueError("undefined contig mapped")

# 
falsePos, falseNeg, tru_pos = 0, 0, 0
for cg in crofts_genomes:
    this_lib = snp_df[snp_df['mapped_lib'] == cg]
    positiveHits = this_lib[this_lib['ref_assem'] == 'CG8421']
    print(cg)
    questionableHits = this_lib.copy()[this_lib['ref_assem'] == 'CG8421_sim']
    questionableHits['contig'] = questionableHits['CHROM'].apply(lambda x: x.split("-")[1])
    questionableHits['real_pos'] = questionableHits[['contig', 'POS']].apply(convert_sim_locs, axis=1)
    print(questionableHits[['POS', 'real_pos', 'CG8421_POS', 'primer_match']])
    questionableHits.sort_values(by='real_pos', inplace=True)
    hitpool = set(positiveHits.index)
    sourcepool = set(questionableHits.index)
    for qh in questionableHits.index:
        hit_dist = abs(positiveHits.loc[hitpool, 'POS'] - questionableHits.loc[qh, 'real_pos'])
        closest_var = hit_dist[hit_dist == hit_dist.min()]
        if len(closest_var) > 0:
            closest_pos = positiveHits.loc[closest_var.index[0], 'POS']
            other_candidates = [abs(questionableHits.loc[sp, 'real_pos']-closest_pos) for sp in sourcepool]
            print(len(sourcepool), len(hitpool), closest_var.min(), min(other_candidates), closest_pos)        
            if (closest_var.min() < 23) and (closest_var.min() <= min(other_candidates)):
                hitpool.remove(closest_var.index[0])
                assert len(closest_var) == 1
                tru_pos += 1
            else:
                falsePos += 1
        else:
            falsePos += 1
    
        sourcepool.remove(qh)
        
    falseNeg += len(hitpool)
    print("{} CG8421 {}, Sim {}, Left Over {}, TP {}, FP {}, FN {}".format(cg, len(questionableHits), len(positiveHits), len(hitpool), 
                                                                           tru_pos, falsePos, falseNeg))

precision = tru_pos/ (tru_pos + falsePos)
recall = tru_pos/ (tru_pos + falseNeg)
f1score=  2*(precision*recall)/(precision+recall)

print("Using assemblies is has a F1 score of {:4f}".format(f1score))

In [ ]:
primer_to_var_region = {}
# map the region back to the variant index
for e_ix, h, mr in e_anchor_df[['headers', 'mutation_rows']].reset_index().values:
    primer_to_var_region[h] = set([e_ix])
    
# map the gene to the variation to the variant index
for h, mr, sr in g_anchor_df[['headers', 'mutation_rows', 'support_rows']].values:
    if mr != []:
        e_ix = list(e_anchor_df[e_anchor_df['mutation_rows'].apply(lambda x: len(set(x).intersection(set(mr))) > 0 )].index)
        primer_to_var_region[h] = set(e_ix)

all_primers_possible = set(primer_to_var_region.keys())
ontarget_hits = good_snps_df['primer_set'].apply(lambda x: len(x.intersection(all_primers_possible)) > 0)
hit_snps = good_snps_df[ ontarget_hits & (good_snps_df['ref_assem'] == 'CG8421')]

snp_locs = {i:set() for i in e_anchor_df.index}
snp_populations = {i:set() for i in e_anchor_df.index}
snp_ref = {i:set() for i in e_anchor_df.index}

for ix in good_snps_df.index:
    if good_snps_df.loc[ix, 'ref_assem'] != 'CG8421_sim':
        these_matches = good_snps_df.loc[ix, 'primer_set']
        these_anchors = [primer_to_var_region[i] for i in these_matches if i in primer_to_var_region.keys()]
        these_anchors = set([i for j in these_anchors for i in j])
        for anch in these_anchors:
            snp_locs[anch].update([good_snps_df.loc[ix, 'CG8421_POS']])    
            snp_populations[anch].update([good_snps_df.loc[ix, 'mapped_lib']])
            snp_ref[anch].update([good_snps_df.loc[ix, 'ref_assem']])
        
flipped_df['SNPs_positions'] = pd.Series(snp_locs)
flipped_df['SNPd_populations'] = pd.Series(snp_populations)
flipped_df['SNP_references'] = pd.Series(snp_ref)

group_keys = [['crofts_innoc'] , primary_crofts, relapse_crofts, peru_relapse]
group_names = ['Innoc', 'Crofts_Primary', 'Crofts_Relapse', 'Iquitos_Relapse']
new_col_names = [gn+"_w_Variant" for gn in group_names]
new_col_names2 = [gn+"_wo_Variant" for gn in group_names]

for gk, ncn, ncn2 in zip(group_keys, new_col_names, new_col_names2):
    flipped_df[ncn2] = pd.Series({k:len(set(gk) - set(v)) for k, v in snp_populations.items()})
    flipped_df[ncn] = pd.Series({k:len(set(gk).intersection(set(v))) for k, v in snp_populations.items()})

expectation_cols = ['Matched_to_Crofts_Data', 'Missing_in_our_Data', 'Misreported_by_them']
for ec in expectation_cols:
    flipped_df[ec] = pd.Series(index=flipped_df.index)

for ix in flipped_df.index:
    snpd_pops = flipped_df.loc[ix, 'SNPd_populations']
    pops_expctd = e_anchor_df.loc[ix, 'Samps_w_Mutation']
    pops_not_expctd = e_anchor_df.loc[ix, 'Samps_off_Target']
    flipped_df.loc[ix, expectation_cols[0]] = len(pops_expctd.intersection(snpd_pops))
    flipped_df.loc[ix, expectation_cols[1]] = len(pops_expctd - snpd_pops)
    flipped_df.loc[ix, expectation_cols[2]] = len(pops_not_expctd.intersection(snpd_pops))

print(flipped_df[expectation_cols].sum())

flipped_df.to_csv('/Volumes/KeithSSD/variant-calling/misc_data/Region_and_Snp_Data.txt', sep="\t")

#valid_snps = hit_snps[['mapped_lib', 'primer_match', 'ref_assem']].groupby(['mapped_lib', 'primer_match'])
#sim_and_not = lambda x: 'CG8421' in list(x) and 'CG8421_sim' in list(x)
#print(valid_snps.agg(set))#.astype(int).sum().values[0])

disp_cols = sorted(['SNPs_positions', 'SNPd_populations', 'SNP_references']+new_col_names + new_col_names2)
flipped_df[disp_cols]


In [ ]:
good_snps_df[good_snps_df['mapped_lib'] == 'crofts_1D2'].sort_values(by='POS')

In [ ]:
print(e_anchor_df.columns)